# Use pre-trained network and append new head

We will use a CNN pre-trained on the Imagenet challenge and append a new "head" to it. 

We will "freeze" the convolutional layers in order to only learn the weights of our new head.

## Create resnet50 body

In [1]:
from resnet50_GAP import ResNet50_GAP

Using TensorFlow backend.


Download the weights from [this link](https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5)

In [2]:
# edit this to the path where you saved the downloaded weights
weights_path = "D:/GitHub/models/resnet50_body.h5"

You can increase the input_shape (probably will lead to better accuracy) according to your GPU capacity.

In [3]:
%%time
body = ResNet50_GAP(input_shape=(300,300,3), weights_path=weights_path)

Wall time: 15.4 s


## Freeze all body layers

In [4]:
for layer in body.layers:
    layer.trainable = False

## Build the head

In [5]:
from keras.models import Model
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [6]:
head = body.output
head = Dense(2, activation="softmax")(head)

## Create model from body+head

In [7]:
model = Model(body.input, head)

## Train / Val generators with augmentation

You can try different augmentations here.

In [8]:
aug_gen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range=0.05,
                             fill_mode="constant",
                             channel_shift_range=10,
                             rotation_range=5,
                             width_shift_range=0.05,
                             height_shift_range=0.05)

Change the `batch_size` according to your GPU capacity.

In [10]:
TRAIN_PATH = "D:/GitHub/Kaggle/redux/train"
VALID_PATH = "D:/GitHub/Kaggle/redux/train"

In [11]:
train_batches = aug_gen.flow_from_directory(TRAIN_PATH, model.input_shape[1:3],
                                                      shuffle=True, batch_size=8)
val_batches = aug_gen.flow_from_directory(VALID_PATH, model.input_shape[1:3],
                                                      shuffle=True, batch_size=8)

Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


## Train the model

Create a checkpoint to save the weights of the best model.

In [12]:
cb = [ModelCheckpoint("best_model_aug.h5", save_best_only=True, save_weights_only=True)]

In [13]:
model.compile(Adam(), "categorical_crossentropy")

Change the number of epochs you want to train.

In [13]:
model.fit_generator(train_batches, train_batches.samples // train_batches.batch_size, 
                    epochs=3, 
                    callbacks=cb,
                    validation_data=val_batches, nb_val_samples=(val_batches.samples //
                   )

Epoch 1/3
23000/23000 [==============================] - 844s - loss: 0.1448 - val_loss: 0.0548
Epoch 2/3
23000/23000 [==============================] - 833s - loss: 0.1274 - val_loss: 0.0513
Epoch 3/3
23000/23000 [==============================] - 831s - loss: 0.1225 - val_loss: 0.0475
